In [1]:
#%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
import pandas as pd
%jsroot on

Welcome to JupyROOT 6.30/06


In [2]:
import multiprocessing as mp

mp.cpu_count()

16

In [3]:
iOption0 = 4 # 0: e, 2: e+, 2: pi, 3: phi, 4: b, 5: c, 6: g0, 7: g1, 8: g2, 9: gee

In [4]:
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,95,0,20,20,40,40,60,60,100]
N_centr = int(len(central_bins)/2)
pt_boarders = [0.2,0.3,0.4,0.5,0.6,0.7,0.9,1.1,1.4,1.9,2.5,3.5,5.0]
N_pt = len(pt_boarders)-1
file_path="/home/yoren/bnl/PHENIX/ee/AuAu/"
file_names=["my-50M_electron_embed_single_new_04-10_v0.root","my-50M_positron_embed_single_new_04-10_v0.root","my-50M_piminus_embed_single_new_04-10_v0.root","my-50M_phi_ee_embed_helios_00-10_v0.root",\
            "my-10M_bbbar_pythia_v0.root","my-50M_ccbar_pythia_v0.root",\
            "my-100M_pi0_gg_embed_helios_new_04-6_v0.root","my-100M_pi0_gg_embed_helios_new_04-6_v1.root",\
            "my-100M_pi0_gg_embed_helios_new_04-6_v2.root","my-50M_pi0_gee_embed_helios_new_04-10_v0.root",\
            "my-10M_bbbar_pythia_v7.root","my-100M_piminus_single_hagedorn_04-5_v0.root"]
code_path="../AnaTrain/Run14AuAuLeptonComby/"
file_name=file_names[iOption0]
outfilenames= ['electron_hits','positron_hits','piminus_new0','phi_new0','bbbar_hits','ccbar_hits','photon_new0','photon_new1','photon_new2','dalitz_new0','pionminus']#output
outfilepath = "/home/yoren/bnl/PHENIX/ee/ML/"
N_start, N_end = 0e6, 25e6
N_threads = 1
filename=outfilenames[iOption0]
part_name = ["e^{+}","e^{-}"]

In [5]:
root.gSystem.Load(code_path+"MyML_C.so")
root.gSystem.Load(code_path+"MyEvent_C.so")

0

In [6]:
root.gROOT.ProcessLine(
f"TFile *input = new TFile(\"{file_path+file_name}\", \"READ\");\
TTree *T = (TTree *)input->Get(\"tree\");\
MyDileptonAnalysis::MyEvent *myevent = 0;\
TBranch *br = T->GetBranch(\"MyEvent\");\
br->SetAddress(&myevent);\
MyDileptonAnalysis::MyEventContainer *event_container = new MyDileptonAnalysis::MyEventContainer();\
event_container->InitEvent();\
event_container->GetHistsFromFile(\"{code_path}field_map.root\");\
"
); 
from ROOT import myevent, br, event_container

File opened at ../AnaTrain/Run14AuAuLeptonComby/field_map.root


mcid: 2 - positron, 3 - electron, 8 - pion+, 9 - pion minus

In [7]:
geant3dict = {
  2: "positron",
  3: "electron",
  5: "+muon",
  6: "-muon",
  8: "+pion",
  9: "-pion",
  11: "+kaon",
  12: "-kaon",
  14: "proton",
  15: "antiproton"
}

In [8]:
if_print = True
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0
soso_event_counter = 0 
for ievent in range(N_real_ev):
    if good_event_counter>200: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if myevent.GetNtrack()<2: continue
            
    n_hits = myevent.GetNVTXhit()
    jhit=0
    for ihit in range(n_hits):
        if myevent.GetVTXHitEntry(jhit).GetSensor() == 1 :
            myevent.RemoveVTXHitEntry(jhit)
        else: jhit+=1
    
    n_hits = myevent.GetNVTXhit()
    for ihit in range(n_hits):
        for jhit in range(ihit+1,n_hits):
            if myevent.GetVTXHitEntry(ihit).GetXHit() == myevent.GetVTXHitEntry(jhit).GetXHit() and myevent.GetVTXHitEntry(ihit).GetYHit() == myevent.GetVTXHitEntry(jhit).GetYHit()\
            and myevent.GetVTXHitEntry(ihit).GetZHit() == myevent.GetVTXHitEntry(jhit).GetZHit():
                print("kek:", myevent.GetVTXHitEntry(ihit).GetClustId(), myevent.GetVTXHitEntry(jhit).GetClustId(), myevent.GetVTXHitEntry(ihit).GetSensor(), myevent.GetVTXHitEntry(jhit).GetSensor())
    
    event_container.correct_beam_offset()
    event_container.Associate_Hits_to_Leptons(5,5,5,0,1)
    for itrack in range(myevent.GetNtrack()):
        if myevent.GetEntry(itrack).GetPtPrime()>0.4 and myevent.GetEntry(itrack).GetTrkQuality()>62 and myevent.GetEntry(itrack).GetMcId()>1 and myevent.GetEntry(itrack).GetMcId()<10:
            soso_event_counter+=1
    do_continue = False
    if event_container.GetNBDThit()>0:
        do_continue = True
    if not do_continue:
        continue
    good_event_counter+=event_container.GetNBDThit()
    if if_print : print(f"Nevent: {ievent}, centrality: {myevent.GetCentrality()}, N VTX hit: {myevent.GetNVTXhit()}, Ntracks: {myevent.GetNtrack()}, NBDTtracks: {event_container.GetNBDThit()}") 

    for ipart in range(event_container.GetNBDThit()):
        if if_print : print("N hits:",  event_container.GetBDTHitEntry(ipart).GetNBDThit(), ", pt:", round(event_container.GetBDTHitEntry(ipart).GetPt(),3),\
              ", id:",  geant3dict[myevent.GetEntry(ipart).GetMcId()])
        for ibdthit in range(event_container.GetBDTHitEntry(ipart).GetNBDThit()):
            bdthit = event_container.GetBDTHitEntry(ipart).GetBDTHitEntry(ibdthit)
            if if_print : print(ibdthit, round(bdthit.Getsdphi(0),3),",",round(bdthit.Getsdphi(1),3),",",round(bdthit.Getsdphi(2),3),",",round(bdthit.Getsdphi(3),3),";",\
                                bdthit.GetSecondHitPhiR(1), bdthit.GetSecondHitPhiL(1), bdthit.GetSecondHitTheR(1), bdthit.GetSecondHitTheL(1), bdthit.GetSecondHitPhiR(1,1), bdthit.GetSecondHitPhiL(1,1), bdthit.GetSecondHitTheR(1,1), bdthit.GetSecondHitTheL(1,1),\
                                bdthit.GetSecondHitPhiR(2), bdthit.GetSecondHitPhiL(2), bdthit.GetSecondHitTheR(2), bdthit.GetSecondHitTheL(2), bdthit.GetSecondHitPhiR(2,1), bdthit.GetSecondHitPhiL(2,1), bdthit.GetSecondHitTheR(2,1), bdthit.GetSecondHitTheL(2,1),\
                                bdthit.GetSecondHitPhiR(3), bdthit.GetSecondHitPhiL(3), bdthit.GetSecondHitTheR(3), bdthit.GetSecondHitTheL(3), bdthit.GetSecondHitPhiR(3,1), bdthit.GetSecondHitPhiL(3,1), bdthit.GetSecondHitTheR(3,1), bdthit.GetSecondHitTheL(3,1),\
                  round(bdthit.GetIsTrue(0),3),",",round(bdthit.GetIsTrue(1),3),",",round(bdthit.GetIsTrue(2),3),",",round(bdthit.GetIsTrue(3),3))
    for ipart in range(myevent.GetNgentrack()):
        if if_print : print(myevent.GetGenTrack(ipart).GetPt())

9989205
Nevent: 38, centrality: 30.0, N VTX hit: 25, Ntracks: 2, NBDTtracks: 2
N hits: 1 , pt: 0.313 , id: electron
0 -1.289 , -1.657 , -2.905 , -10.0 ; -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -16.063749313354492 8.75738525390625 8.543460845947266 -10.383811950683594 37.10176086425781 28.194480895996094 4.956390857696533 5.14517879486084 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 1.0 , 1.0 , 1.0 , 0.0
N hits: 3 , pt: 0.848 , id: positron
0 0.192 , -0.742 , -1.212 , -10.0 ; -13.36892318725586 53.29359817504883 2.429246187210083 6.303288459777832 -18.090349197387695 -99.0 3.1670925617218018 -99.0 -1.4710309505462646 4.094349384307861 5.275539398193359 10.754600524902344 10.652849197387695 -99.0 9.112237930297852 -99.0 -99.0 21.765689849853516 -99.0 11.819889068603516 -99.0 29.448436737060547 -99.0 3.070648431777954 1.0 , 1.0 , 1.0 , 0.0
1 0.192 , -0.742 , 0.124 , -1.279 ; -13.36892318725586 53.29359817504883 2.429246187210083 6.303288459777832 -18.090349197387695 -99.0 3.167

In [9]:
print(good_event_counter,soso_event_counter, ievent)

201 271 3793


In [10]:
def write_TTree_to_CVS(N_start_loc, N_end_loc, iter = 0):
    data_array = []
    good_event_counter = 0
    soso_event_counter = 0 
    for ievent in range(int(N_start_loc), min(int(N_end_loc),N_real_ev)):
        if (ievent - N_start_loc) % 1e5 == 0: print("N of events:", ievent - N_start_loc, iter)
        if good_event_counter>100000000: break
        myevent.ClearEvent()
        br.GetEntry(ievent)
        event_container.SetEvent(myevent)
        if not myevent.GetNtrack(): continue
        if myevent.GetNtrack() > myevent.GetNgentrack(): continue

        n_hits = myevent.GetNVTXhit()
        jhit=0
        for ihit in range(n_hits):
            if myevent.GetVTXHitEntry(jhit).GetSensor() == 1 :
                myevent.RemoveVTXHitEntry(jhit)
            else: jhit+=1

        event_container.correct_beam_offset()
        event_container.Associate_Hits_to_Leptons(5,5,5,0,1)
        if event_container.GetNBDThit() > myevent.GetNgentrack(): continue
        soso_event_counter+=1
        do_continue = False
        for ipart in range(myevent.GetNtrack()):
            if event_container.GetNBDThit()>0:
                do_continue = True
        if not do_continue:
            continue
        good_event_counter+=1
        sdphi_0,sdphi_1 =  myevent.GetEntry(0).GetMinsDphi(0), myevent.GetEntry(0).GetMinsDphi(1)
        for ipart in range(event_container.GetNBDThit()):
            BDTpart  = event_container.GetBDTHitEntry(ipart)
            gentrack = myevent.GetGenTrack(0)
            for ibdthit in range(BDTpart.GetNBDThit()):
                bdthit = event_container.GetBDTHitEntry(ipart).GetBDTHitEntry(ibdthit)
                if abs(sdphi_0- bdthit.Getsdphi(0))>0.01 or abs(sdphi_1- bdthit.Getsdphi(1))>0.01: 
                    continue
                data_array.append([BDTpart.GetNBDThit(), BDTpart.GetPt(), BDTpart.GetPhi0(), BDTpart.GetThe0(), BDTpart.GetPhiDC(), BDTpart.GetZDC(), \
                                   BDTpart.GetAlpha(), BDTpart.GetEcore(), BDTpart.GetCentrality(), BDTpart.GetCharge(), BDTpart.GetArm(),\
                                   bdthit.GetReconPt(), bdthit.GetReconPhi0(), bdthit.GetReconThe0(),\
                                   gentrack.GetPx(), gentrack.GetPy(), gentrack.GetPz(), gentrack.GetVx()/10**15-myevent.GetPreciseX(), gentrack.GetVy()/10**15-myevent.GetPreciseY(), gentrack.GetVz()/10**15-myevent.GetPreciseZ(),\
                                   bdthit.Getsdphi(0), bdthit.Getsdthe(0), bdthit.GetIsTrue(0), bdthit.GetSecondHitPhiR(0), bdthit.GetSecondHitPhiL(0), bdthit.GetSecondHitTheR(0), bdthit.GetSecondHitTheL(0), bdthit.GetSecondHitPhiR(0,1), bdthit.GetSecondHitPhiL(0,1), bdthit.GetSecondHitTheR(0,1), bdthit.GetSecondHitTheL(0,1),\
                                   bdthit.Getsdphi(1), bdthit.Getsdthe(1), bdthit.GetIsTrue(1), bdthit.GetSecondHitPhiR(1), bdthit.GetSecondHitPhiL(1), bdthit.GetSecondHitTheR(1), bdthit.GetSecondHitTheL(1), bdthit.GetSecondHitPhiR(1,1), bdthit.GetSecondHitPhiL(1,1), bdthit.GetSecondHitTheR(1,1), bdthit.GetSecondHitTheL(1,1),\
                                   bdthit.Getsdphi(2), bdthit.Getsdthe(2), bdthit.GetIsTrue(2), bdthit.GetSecondHitPhiR(2), bdthit.GetSecondHitPhiL(2), bdthit.GetSecondHitTheR(2), bdthit.GetSecondHitTheL(2), bdthit.GetSecondHitPhiR(2,1), bdthit.GetSecondHitPhiL(2,1), bdthit.GetSecondHitTheR(2,1), bdthit.GetSecondHitTheL(2,1),\
                                   bdthit.Getsdphi(3), bdthit.Getsdthe(3), bdthit.GetIsTrue(3), bdthit.GetSecondHitPhiR(3), bdthit.GetSecondHitPhiL(3), bdthit.GetSecondHitTheR(3), bdthit.GetSecondHitTheL(3), bdthit.GetSecondHitPhiR(3,1), bdthit.GetSecondHitPhiL(3,1), bdthit.GetSecondHitTheR(3,1), bdthit.GetSecondHitTheL(3,1),\
                                    ])
                #print(BDTpart.GetPt()-(gentrack.GetPx()**2+gentrack.GetPy()**2)**0.5)

    data = pd.DataFrame(data_array, columns=('NBDThit', 'Pt', 'Phi0', 'The0', 'PhiDC', 'ZDC', 'Alpha', 'Ecore', 'Centrality', 'Charge', 'Arm',\
                                             'reconpt','reconphi0','reconthe0','px','py','pz','vx','vy','vz',\
                                             'sdphi0','sdthe0','IsTrue0','SecondHitPhiR0','SecondHitPhiL0','SecondHitTheR0','SecondHitTheL0','SecondHitPhiR01','SecondHitPhiL01','SecondHitTheR01','SecondHitTheL01',\
                                             'sdphi1','sdthe1','IsTrue1','SecondHitPhiR1','SecondHitPhiL1','SecondHitTheR1','SecondHitTheL1','SecondHitPhiR11','SecondHitPhiL11','SecondHitTheR11','SecondHitTheL11',\
                                             'sdphi2','sdthe2','IsTrue2','SecondHitPhiR2','SecondHitPhiL2','SecondHitTheR2','SecondHitTheL2','SecondHitPhiR21','SecondHitPhiL21','SecondHitTheR21','SecondHitTheL21',\
                                             'sdphi3','sdthe3','IsTrue3','SecondHitPhiR3','SecondHitPhiL3','SecondHitTheR3','SecondHitTheL3','SecondHitPhiR31','SecondHitPhiL31','SecondHitTheR31','SecondHitTheL31'))
    
    float_columns=('Pt', 'Phi0', 'The0', 'PhiDC', 'ZDC', 'Alpha', 'Ecore',\
                   'reconpt','reconphi0','reconthe0','px','py','pz','vx','vy','vz',\
                   'sdphi0','sdthe0','SecondHitPhiR0','SecondHitPhiL0','SecondHitTheR0','SecondHitTheL0','SecondHitPhiR01','SecondHitPhiL01','SecondHitTheR01','SecondHitTheL01',\
                   'sdphi1','sdthe1','SecondHitPhiR1','SecondHitPhiL1','SecondHitTheR1','SecondHitTheL1','SecondHitPhiR11','SecondHitPhiL11','SecondHitTheR11','SecondHitTheL11',\
                   'sdphi2','sdthe2','SecondHitPhiR2','SecondHitPhiL2','SecondHitTheR2','SecondHitTheL2','SecondHitPhiR21','SecondHitPhiL21','SecondHitTheR21','SecondHitTheL21',\
                   'sdphi3','sdthe3','SecondHitPhiR3','SecondHitPhiL3','SecondHitTheR3','SecondHitTheL3','SecondHitPhiR31','SecondHitPhiL31','SecondHitTheR31','SecondHitTheL31')
    int_columns=('NBDThit','Centrality', 'Charge', 'Arm','IsTrue0','IsTrue1','IsTrue2','IsTrue3')
    
    for float_column in float_columns:
        data[float_column] = data[float_column].astype('float32')
    for int_column in int_columns:
        data[int_column] = data[int_column].astype('int32')
    
    print(data.size, data.shape, good_event_counter, soso_event_counter, ievent+1, ievent - N_start_loc+1, iter)
    filepath = outfilepath+filename+str(iter)
    compression_options = dict(method='zip', archive_name=f'{filename}{iter}.csv')

    data.to_csv(f'{filepath}.zip', compression=compression_options, index=False)

    return [[data.size, data.shape, good_event_counter,soso_event_counter, ievent+1, ievent - N_start_loc+1, iter],data.head(),f'{filepath}.zip']

In [11]:
out = write_TTree_to_CVS(0,100000)

N of events: 0 0
1660480 (25945, 64) 19327 43166 100000 100000 0


In [12]:
print(out[0])
out[1]

[1660480, (25945, 64), 19327, 43166, 100000, 100000, 0]


,NBDThit,Pt,Phi0,The0,PhiDC,ZDC,Alpha,Ecore,Centrality,Charge,...,sdthe3,IsTrue3,SecondHitPhiR3,SecondHitPhiL3,SecondHitTheR3,SecondHitTheL3,SecondHitPhiR31,SecondHitPhiL31,SecondHitTheR31,SecondHitTheL31
0,1,0.835107,0.809616,1.475586,0.564941,20.725420,0.122131,0.872146,3,-1,...,-10.00000,0,-99.000000,0.699666,-99.000000,1.505101,-99.000000,0.699666,-99.00000,15.395395
1,1,0.403897,3.194386,1.278320,2.669922,65.315796,0.269043,0.370708,55,-1,...,-10.00000,0,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.00000,-99.000000
2,1,0.415572,0.037693,1.844727,-0.493408,-58.439381,0.266113,-9999.000000,82,-1,...,-10.00000,0,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.00000,-99.000000
3,1,1.236941,2.127112,1.498047,2.291016,16.038572,-0.083069,1.187184,9,1,...,-1.41333,0,7.106871,-99.000000,6.218014,-99.000000,-16.168034,-99.000000,2.49828,-99.000000
4,3,0.692949,2.991292,1.271484,2.677734,67.000252,0.161743,0.686260,41,-1,...,-10.00000,0,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.00000,-99.000000


In [13]:
def write_TTree_to_CVS_MP(iter):
    delta  = int((N_end - N_start)/N_threads)
    N_start_loc, N_end_loc = N_start + delta*iter, N_start + delta*(iter+1)
    output =  write_TTree_to_CVS(N_start_loc,N_end_loc,iter)
    return output


In [14]:
Ntr = N_threads

pool = mp.Pool(Ntr)
with pool:
    output_array = pool.map(write_TTree_to_CVS_MP,  [i for i in range(Ntr)])
pool.close()

N of events: 0.0 0
N of events: 100000.0 0
N of events: 200000.0 0
N of events: 300000.0 0
N of events: 400000.0 0
N of events: 500000.0 0
N of events: 600000.0 0
N of events: 700000.0 0
N of events: 800000.0 0
N of events: 900000.0 0
N of events: 1000000.0 0
N of events: 1100000.0 0
N of events: 1200000.0 0
N of events: 1300000.0 0
N of events: 1400000.0 0
N of events: 1500000.0 0
N of events: 1600000.0 0
N of events: 1700000.0 0
N of events: 1800000.0 0
N of events: 1900000.0 0
N of events: 2000000.0 0
N of events: 2100000.0 0
N of events: 2200000.0 0
N of events: 2300000.0 0
N of events: 2400000.0 0
N of events: 2500000.0 0
N of events: 2600000.0 0
N of events: 2700000.0 0
N of events: 2800000.0 0
N of events: 2900000.0 0
N of events: 3000000.0 0
N of events: 3100000.0 0
N of events: 3200000.0 0
N of events: 3300000.0 0
N of events: 3400000.0 0
N of events: 3500000.0 0
N of events: 3600000.0 0
N of events: 3700000.0 0
N of events: 3800000.0 0
N of events: 3900000.0 0
N of events: 40

In [15]:
zips = []
for zipname in output_array:
    zips.append(zipname[2])

In [16]:
print(zips)

['/home/yoren/bnl/PHENIX/ee/ML/bbbar_hits0.zip']


In [17]:
import zipfile as z
import os

In [18]:
with z.ZipFile(zips[0], 'a') as z1:
    for fname in zips[1:]:
        zf = z.ZipFile(fname, 'r')
        for n in zf.namelist():
            z1.writestr(n, zf.open(n).read())
for izip in range(1,len(zips)):    
    os.remove(zips[izip])


In [19]:
if False:
    data1 = pd.read_csv("/home/yoren/bnl/PHENIX/ee/ML/"+'ccbar0.csv')